📘 1️⃣ Project Introduction

# 🌍 Ecosense AI — Intelligent Carbon Footprint Estimator & Green Action Recommender

Ecosense AI is an interactive sustainability assistant that:

- Accurately estimates a user’s carbon footprint  
- Uses scientific emission factors  
- Loads Mistral-7B Instruct (4-bit optimized)  
- Analyzes user lifestyle patterns  
- Generates personalized recommendations  
- Classifies carbon footprint (LOW / MODERATE / HIGH)  
- Provides a rule-based fallback if AI is unavailable  
- Visualizes results with interactive charts  
- Stores session data using Flask  

This notebook installs dependencies, prepares the environment, removes sensitive tokens, explains how to insert your own, and launches the complete web application using ngrok.


📘 2️⃣ Install Dependencies

This step installs all required packages:

- Flask → Web framework  
- pyngrok → Public URL exposure  
- transformers → Load the Mistral LLM  
- accelerate + bitsandbytes → 4-bit inference (faster & memory-efficient)  
- sentencepiece + safetensors → Tokenization & safe model file loading  
- gradio → (Optional) UI testing  
- Folder creation for templates & static assets  

No changes needed here.


In [ ]:
# Run this cell first in Colab
!pip install -q flask pyngrok transformers accelerate bitsandbytes sentencepiece safetensors gradio==4.38.1

# create folders
!mkdir -p templates static

# set your HF token in Colab runtime BEFORE running heavy model-load step:
# Example (in a secure way):



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.1/318.1 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.1/118.1 kB 4.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yfinance 0.2.66 requires websockets>=13.0, but you have websockets 11.0.3 which is incompatible.
google-adk 1.17.0 requires websockets<16.0.0,>=15.0.1, but you have websockets 11.0.3 which is incompatible.
google-genai 1.49.0 requires websockets<15.1.0,>=13.0.0, but you have websockets 11.0.3 which is incompatible.
dataproc-spark-connect 0.8.3 requires websockets>=14.0, but you have websockets 11.0.3 which is incompatible.


📘 3️⃣ Hugging Face Authentication

# Hugging Face Login — User Action Required

Your notebook originally contained an HF token.  
It has been removed for security.

### Generate your own Hugging Face token:
1. Visit https://huggingface.co/settings/tokens  
2. Click **New Token**  
3. Set permission → Read  
4. Copy the token  
5. Paste it into the notebook:

login(token="YOUR_HF_TOKEN_HERE")

⚠️ Never commit your token to GitHub or share it publicly.


In [ ]:
# ===============================
# 2️⃣ Hugging Face Authentication
# ===============================
from huggingface_hub import login
login(token="YOUR_HF_TOKEN_HERE")   # replace with your token


📘 4️⃣ Flask Backend (app.py)

The Flask application handles:

- Web routing (/, /estimate, /result, /suggestions, /insights)  
- Form collection and validation  
- Carbon footprint calculation using scientific factors  
- JSON-based LLM prompt building  
- Secure model loading using 4-bit quantization  
- Session storage for results  
- Rule-based fallback if LLM fails  
- Structured AI output passed to HTML templates  

The backend also uses:
- functools.lru_cache → to load the LLM only once  
- torch → GPU/CPU auto detection  
- Mistral-7B-Instruct → Efficient generative model  


📘 5️⃣ Carbon Footprint Calculation

Ecosense AI computes emissions using verified factors:

- Electricity (kWh)
- Cooking fuel type
- Car travel (km + fuel type)
- Public transport usage
- Diet type (vegetarian / mixed / high meat)
- Waste generation & disposal method
- Water consumption

The output includes a detailed breakdown:
{
 "household_energy": …,
 "cooking": …,
 "car_travel": …,
 ...
 "total": …
}

This ensures scientifically grounded CO₂ estimation.


📘 6️⃣ LLM Behavior & JSON Output

The LLM is instructed to output STRICT JSON with:

- classification
- message
- recommendations list
- top_contributors
- confidence

If JSON parsing fails:
- AI raw output is shown
- Rule-based fallback recommendations are activated

This ensures reliability even when the model responds unpredictably.


In [ ]:
%%writefile app.py
import os
import json
import traceback
import functools
from datetime import datetime

from flask import (
    Flask, render_template, request,
    redirect, url_for, session
)

import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    pipeline
)

# ==========================================================
# FLASK APP CONFIG
# ==========================================================
app = Flask(__name__, static_folder="static", template_folder="templates")
app.secret_key = "ecosense-secret-key-123"   # Needed for session storage


# ==========================================================
# MODEL CONFIG
# ==========================================================
MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.3"
HF_TOKEN = os.environ.get("HF_TOKEN", None)

GEN_PARAMS = {
    "temperature": 0.4,
    "top_p": 0.95,
    "do_sample": True,
    "max_new_tokens": 350
}

# ==========================================================
# EMISSION FACTORS
# ==========================================================
EMISSION_FACTORS = {
    "electricity_kwh": 0.92,
    "cooking_fuel": {
        "electricity": 0.92,
        "natural_gas": 2.05,
        "liquefied_petroleum_gas": 2.96,
        "wood": 1.80,
        "other": 1.5,
    },
    "car_travel_km": {
        "petrol": 0.192,
        "diesel": 0.171,
        "electric": 0.053,
        "hybrid": 0.110,
    },
    "public_transport_km": 0.05,
    "diet": {
        "vegetarian": 3.8,
        "mixed": 5.0,
        "high_meat": 8.0,
    },
    "waste_kg": {
        "recycling": 0.02,
        "compost": 0.04,
        "landfill": 0.45,
    },
    "water_liters": 0.0003
}

def safe_float(x):
    try:
        return float(x or 0)
    except:
        return 0.0


# ==========================================================
# CARBON FOOTPRINT CALCULATION
# ==========================================================
def calculate_footprint(data):
    energy_kwh = safe_float(data.get("electricity_kwh"))
    cooking_fuel = data.get("cooking_fuel", "electricity")
    car_km = safe_float(data.get("car_km"))
    car_fuel = data.get("car_fuel", "petrol")
    public_km = safe_float(data.get("public_km"))
    diet_type = data.get("diet", "mixed")
    waste_kg = safe_float(data.get("waste_kg"))
    waste_type = data.get("waste_type", "landfill")
    water_liters = safe_float(data.get("water_liters"))

    energy_emission = energy_kwh * EMISSION_FACTORS["electricity_kwh"]
    cooking_emission = EMISSION_FACTORS["cooking_fuel"][cooking_fuel] * energy_kwh
    car_emission = car_km * EMISSION_FACTORS["car_travel_km"][car_fuel]
    public_emission = public_km * EMISSION_FACTORS["public_transport_km"]
    diet_emission = EMISSION_FACTORS["diet"][diet_type]
    waste_emission = waste_kg * EMISSION_FACTORS["waste_kg"][waste_type]
    water_emission = water_liters * EMISSION_FACTORS["water_liters"]

    total = (
        energy_emission + cooking_emission + car_emission +
        public_emission + diet_emission + waste_emission + water_emission
    )

    return {
        "household_energy": round(energy_emission, 2),
        "cooking": round(cooking_emission, 2),
        "car_travel": round(car_emission, 2),
        "public_transport": round(public_emission, 2),
        "diet_daily": round(diet_emission, 2),
        "waste": round(waste_emission, 2),
        "water": round(water_emission, 2),
        "total": round(total, 2)
    }


# ==========================================================
# RULE-BASED FALLBACK (only if LLM fails)
# ==========================================================
def generate_rule_recommendations(data):
    recs = []
    car_km = safe_float(data.get("car_km"))

    if car_km > 50:
        recs.append({
            "action": "Reduce weekly car travel",
            "rationale": "High emissions from frequent driving.",
            "est_weekly_savings_kg": round((car_km - 50) * 0.171, 2),
            "difficulty": "medium",
            "timeframe": "weeks"
        })

    if data.get("diet") == "high_meat":
        recs.append({
            "action": "Reduce red meat consumption",
            "rationale": "Meat-heavy diets generate high emissions.",
            "est_weekly_savings_kg": 5.0,
            "difficulty": "medium",
            "timeframe": "weeks"
        })

    if data.get("waste_type") == "landfill":
        recs.append({
            "action": "Switch to recycling/composting",
            "rationale": "Landfill waste produces methane emissions.",
            "est_weekly_savings_kg": 1.5,
            "difficulty": "low",
            "timeframe": "immediate"
        })

    return recs


# ==========================================================
# LOAD LLM — 4-bit Mistral (cached)
# ==========================================================
@functools.lru_cache(maxsize=1)
def load_llm():
    print("🔋 Loading Mistral-7B 4-bit… please wait…")

    bnb = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
    )

    tokenizer = AutoTokenizer.from_pretrained(
        MODEL_NAME,
        token=HF_TOKEN,
        use_fast=True
    )

    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        device_map="auto",
        quantization_config=bnb,
        torch_dtype=torch.float16,
        token=HF_TOKEN
    )

    pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device_map="auto")
    print("✔ Model loaded successfully!")
    return pipe


# ==========================================================
# BUILD LLM PROMPT
# ==========================================================
def build_prompt(user_data, footprint):

    return f"""
You are a helpful carbon footprint expert AI.

Classify user CO₂ level:
- HIGH = total > 800
- MODERATE = 300–800
- LOW = < 300

Respond ONLY in JSON with this structure:

{{
 "classification": "",
 "message": "",
 "recommendations": [
    {{
      "action": "",
      "rationale": "",
      "est_weekly_savings_kg": 0,
      "difficulty": "",
      "timeframe": ""
    }}
 ],
 "top_contributors": [],
 "confidence": 0.0
}}

NO extra text. Output ONLY JSON.

User Input: {json.dumps(user_data)}
Footprint: {json.dumps(footprint)}
"""


# ==========================================================
# CALL LLM
# ==========================================================
def call_llm(pipe, prompt):
    out = pipe(prompt, **GEN_PARAMS)
    raw = out[0]["generated_text"]

    try:
        json_text = raw[raw.find("{"):]
        return json.loads(json_text)
    except:
        return {"raw_text": raw, "error": "json_parse_failed"}


# ==========================================================
# ROUTES
# ==========================================================
@app.route("/")
def home():
    return render_template("index.html")

@app.route("/insights")
def insights():
    return render_template("insights.html")


@app.route("/suggestions")
def suggestions():
    results = session.get("results", None)
    ai = None
    if results:
        ai = results.get("ai")

    return render_template("suggestions.html", ai=ai)



@app.route("/estimate", methods=["POST"])
def estimate():
    # Get user input
    form = request.form.to_dict()

    # Calculate emission
    footprint = calculate_footprint(form)
    prompt = build_prompt(form, footprint)

    # Call LLM safely
    try:
        pipe = load_llm()
        ai = call_llm(pipe, prompt)
    except Exception:
        traceback.print_exc()
        ai = {
            "message": "LLM unavailable, using fallback suggestions.",
            "recommendations": generate_rule_recommendations(form),
            "classification": "UNKNOWN",
            "confidence": 0.3
        }

    # Save to session
    session["results"] = {
        "input": form,
        "footprint": footprint,
        "ai": ai,
        "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    }

    return redirect(url_for("result_page"))


@app.route("/result", methods=["GET"])
def result_page():
    results = session.get("results", None)   # FIXED LINE
    if not results:
        return redirect("/")

    return render_template(
        "result.html",
        breakdown=results.get("footprint"),   # FIXED KEY
        ai=results.get("ai"),
        rules=results.get("ai", {}).get("recommendations"),
        timestamp=results.get("timestamp")
    )




# ==========================================================
# MAIN — PRELOAD MODEL
# ==========================================================
if __name__ == "__main__":
    print("🔥 Starting Ecosense AI Server…")
    print("CUDA:", torch.cuda.is_available())
    load_llm()
    print("✔ Model ready!")
    app.run(host="0.0.0.0", port=8000, debug=False)


Overwriting app.py


📘 7️⃣ Frontend Templates (index.html, result.html, suggestions.html)

These templates provide:

✔ A modern two-column layout  
✔ Styled input form for lifestyle data  
✔ Responsive design  
✔ Results page with:
   - Breakdown stats  
   - Doughnut chart  
   - AI-generated recommendations  
✔ Saved suggestions page  
✔ Navigation bar for easy switching  

Templates are stored in:
templates/


In [ ]:
%%writefile templates/index.html
<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8" />
  <meta name="viewport" content="width=device-width,initial-scale=1" />
  <title>Ecosense AI — Carbon Footprint Estimator</title>

  <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}" />
  <link href="https://fonts.googleapis.com/css2?family=Inter:wght@300;400;600;700;800&display=swap" rel="stylesheet">
</head>

<body>
  <!-- Navigation -->
  <header class="nav">
    <div class="nav-left">
      <span class="logo-icon">🌱</span>
      <div>
        <h1 class="brand">Ecosense AI</h1>
        <div class="tag">Smarter • Greener • Personalized Sustainability</div>
      </div>
    </div>

    <nav class="nav-right">
      <a href="/" class="nav-link active">Home</a>
      <a href="/insights" class="nav-link">Insights</a>
      <a href="/suggestions" class="nav-link">Suggestions</a>
    </nav>
  </header>

  <!-- Main Layout -->
  <main class="main">
    <!-- Left Column (Form Only) -->
    <section class="left-col">
      <div class="card form-card">
        <h2 class="card-heading">Enter Your Lifestyle Data</h2>

        <!-- FORM POSTS DIRECTLY TO /estimate -->
        <form id="carbonForm" method="POST" action="/estimate" autocomplete="off">

          <div class="grid-2">
            <label class="field">
              <span class="field-label">💡 Electricity Use (kWh / month)</span>
              <input type="number" name="electricity_kwh" min="0" required value="300">
            </label>

            <label class="field">
              <span class="field-label">🔥 Cooking Fuel</span>
              <select name="cooking_fuel" required>
                <option value="electricity">Electricity</option>
                <option value="natural_gas">Natural Gas</option>
                <option value="liquefied_petroleum_gas">LPG</option>
                <option value="wood">Wood</option>
                <option value="other">Other</option>
              </select>
            </label>
          </div>

          <div class="grid-2">
            <label class="field">
              <span class="field-label">🚗 Car Travel (km / week)</span>
              <input type="number" name="car_km" min="0" required value="100">
            </label>

            <label class="field">
              <span class="field-label">⛽ Fuel Type</span>
              <select name="car_fuel" required>
                <option value="petrol">Petrol</option>
                <option value="diesel">Diesel</option>
                <option value="electric">Electric</option>
                <option value="hybrid">Hybrid</option>
              </select>
            </label>
          </div>

          <div class="grid-2">
            <label class="field">
              <span class="field-label">🚌 Public Transport (km / week)</span>
              <input type="number" name="public_km" min="0" required value="30">
            </label>

            <label class="field">
              <span class="field-label">🥦 Diet Type</span>
              <select name="diet" required>
                <option value="vegetarian">Vegetarian</option>
                <option value="mixed">Mixed</option>
                <option value="high_meat">High Meat</option>
              </select>
            </label>
          </div>

          <div class="grid-2">
            <label class="field">
              <span class="field-label">🗑️ Waste (kg / week)</span>
              <input type="number" step="0.1" name="waste_kg" min="0" required value="2">
            </label>

            <label class="field">
              <span class="field-label">♻️ Waste Disposal</span>
              <select name="waste_type" required>
                <option value="recycling">Recycling</option>
                <option value="compost">Compost</option>
                <option value="landfill">Landfill</option>
              </select>
            </label>
          </div>

          <label class="field">
            <span class="field-label">💧 Water Usage (liters / month)</span>
            <input type="number" name="water_liters" min="0" required value="8000">
          </label>

          <div class="actions">
            <button class="btn primary" type="submit">Calculate My Carbon Footprint</button>
            <button class="btn ghost" type="reset">Reset</button>
          </div>
        </form>
      </div>

      <!-- Quick Tips -->
      <div class="card tips-card">
        <h3>Quick Tips</h3>
        <ul>
          <li>Use LED bulbs and energy-efficient appliances</li>
          <li>Choose public transport or carpool</li>
          <li>Reduce red meat consumption</li>
          <li>Recycle and compost whenever possible</li>
        </ul>
      </div>
    </section>

    <!-- Right Column (Now simple welcome info) -->
    <section class="right-col">
      <div class="card">
        <h2>Welcome to Ecosense AI 🌍</h2>
        <p style="margin-top:10px; line-height:1.6;">
          This tool helps you accurately calculate your household's carbon footprint
          and provides personalized, AI-driven recommendations to reduce it.
        </p>
        <p style="margin-top:10px;">
          Enter your details on the left, and your customized footprint report
          will appear on the next page.
        </p>
        <div style="margin-top:20px; font-size:14px; color:#345;">
          • Smart CO₂ estimation
          • AI-generated action plan
          • Easy to understand for everyone
        </div>
      </div>
    </section>
  </main>

  <footer class="footer">
    <div>Made with 🌿 • Ecosense AI</div>
    <div class="footer-actions">
      <a href="/insights">Insights</a> •
      <a href="/suggestions">Suggestions</a>
    </div>
  </footer>
</body>
</html>


Overwriting templates/index.html


In [ ]:
%%writefile templates/result.html
<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width,initial-scale=1">
  <title>Ecosense AI — Your Carbon Footprint Report</title>

  <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}" />
  <script src="https://cdn.jsdelivr.net/npm/chart.js@4.4.0/dist/chart.umd.min.js"></script>
</head>

<body>

  <!-- Navigation -->
  <header class="nav small">
    <div class="nav-left">
      <span class="logo-icon">🌱</span>
      <div>
        <h1 class="brand">Ecosense AI</h1>
        <div class="tag">Your Personalized Sustainability Report</div>
      </div>
    </div>
    <nav class="nav-right">
      <a href="/" class="nav-link">Home</a>
      <a href="/result" class="nav-link active">Results</a>
      <a href="/suggestions" class="nav-link">Suggestions</a>
    </nav>
  </header>

  <!-- MAIN CONTENT -->
  <main class="main">

    <!-- LEFT COLUMN -->
    <section class="left-col">
      <div class="card">

        <h2>Your Carbon Footprint Report</h2>
        <p class="small">Generated on {{ timestamp }}</p>

        <!-- TOTAL -->
        <div class="total" style="margin-top:15px;">
          <strong>Total Emissions:</strong>
          <span class="total-val">{{ breakdown.total }} kg CO₂</span>
        </div>

        <!-- BREAKDOWN GRID -->
        <div class="breakdown-grid" style="margin-top:20px;">
          <div class="stat"><div class="stat-label">Household Energy</div><div class="stat-value">{{ breakdown.household_energy }} <span class="unit">kg</span></div></div>
          <div class="stat"><div class="stat-label">Cooking</div><div class="stat-value">{{ breakdown.cooking }} <span class="unit">kg</span></div></div>
          <div class="stat"><div class="stat-label">Car Travel</div><div class="stat-value">{{ breakdown.car_travel }} <span class="unit">kg</span></div></div>
          <div class="stat"><div class="stat-label">Public Transport</div><div class="stat-value">{{ breakdown.public_transport }} <span class="unit">kg</span></div></div>
          <div class="stat"><div class="stat-label">Diet</div><div class="stat-value">{{ breakdown.diet_daily }} <span class="unit">kg</span></div></div>
          <div class="stat"><div class="stat-label">Waste</div><div class="stat-value">{{ breakdown.waste }} <span class="unit">kg</span></div></div>
          <div class="stat"><div class="stat-label">Water</div><div class="stat-value">{{ breakdown.water }} <span class="unit">kg</span></div></div>
        </div>

        <!-- CLASSIFICATION BOX -->
        {% if ai.classification %}
        <div class="classification-box {{ ai.classification.lower() }}" style="margin-top:20px;">
          <div class="class-title">Category: {{ ai.classification }}</div>
          <div class="class-msg">{{ ai.message }}</div>
        </div>
        {% endif %}

        <!-- AI RECOMMENDATIONS -->
        {% if ai.recommendations %}
        <div class="ai-section" style="margin-top:25px;">
          <h3>Personalized Recommendations</h3>

          {% for r in ai.recommendations %}
          <div class="ai-reco">
            <div class="ai-title">{{ loop.index }}. {{ r.action }}</div>
            <div class="ai-body">{{ r.rationale }}</div>
            <div class="ai-meta">
              Weekly Savings: {{ r.est_weekly_savings_kg }} kg •
              Difficulty: {{ r.difficulty }} •
              Timeframe: {{ r.timeframe }}
            </div>
          </div>
          {% endfor %}
        </div>
        {% endif %}

        <!-- RAW AI FALLBACK -->
        {% if ai.raw_text %}
        <div class="ai-section">
          <h3>AI Output (Raw)</h3>
          <pre class="raw-ai">{{ ai.raw_text }}</pre>
        </div>
        {% endif %}

        <!-- RULE BASED FALLBACK -->
        {% if rules %}
        <div class="ai-section">
          <h3>Suggestions</h3>
          {% for r in rules %}
          <div class="ai-reco">
            <div class="ai-title">{{ loop.index }}. {{ r.action }}</div>
            <div class="ai-body">{{ r.rationale }}</div>
            <div class="ai-meta">Estimated Savings: {{ r.est_weekly_savings_kg }} kg</div>
          </div>
          {% endfor %}
        </div>
        {% endif %}

        <!-- EDUCATIONAL TIPS -->
        <div class="edu" style="margin-top:20px;">
          <h3>General Tips</h3>
          <ul>
            <li>Use renewable energy sources when possible.</li>
            <li>Use public transport, carpool or cycle.</li>
            <li>Try plant-based meals more often.</li>
            <li>Recycle & compost to reduce methane emissions.</li>
          </ul>
        </div>

        <div style="margin-top:25px;">
          <a href="/" class="btn primary">Calculate Again</a>
        </div>

      </div>
    </section>

    <!-- RIGHT COLUMN -->
    <section class="right-col">

      <div class="card chart-card">
        <h3>Top Contributors</h3>
        <canvas id="contribChart" height="250"></canvas>
        <div id="chartLegend" class="chart-legend"></div>
      </div>

    </section>

  </main>

  <footer class="footer small">
    <div>Made with 🌿 • Ecosense AI</div>
    <div><a href="/">Back to Home</a></div>
  </footer>

  <!-- Chart Script -->
  <script>
    const items = [
      { label: "Household Energy", value: {{ breakdown.household_energy }} },
      { label: "Cooking", value: {{ breakdown.cooking }} },
      { label: "Car Travel", value: {{ breakdown.car_travel }} },
      { label: "Public Transport", value: {{ breakdown.public_transport }} },
      { label: "Diet", value: {{ breakdown.diet_daily }} },
      { label: "Waste", value: {{ breakdown.waste }} },
      { label: "Water", value: {{ breakdown.water }} }
    ];

    const labels = items.map(i => i.label);
    const values = items.map(i => i.value);
    const bg = labels.map((_, i) => `hsl(${(i*40)%360} 70% 55% / 0.85)`);

    const ctx = document.getElementById('contribChart').getContext('2d');

    new Chart(ctx, {
      type: 'doughnut',
      data: {
        labels,
        datasets: [{ data: values, backgroundColor: bg }]
      },
      options: {
        plugins: { legend: { display: true, position: 'bottom' } },
        maintainAspectRatio: false,
        cutout: '50%'
      }
    });

    document.getElementById('chartLegend').innerHTML =
      labels.map((l, i) =>
        `<span class="lg-item">
          <span class="dot" style="background:${bg[i]}"></span>
          ${l}: ${values[i]}
        </span>`
      ).join('');
  </script>

</body>
</html>


Overwriting templates/result.html


In [ ]:
%%writefile templates/suggestions.html
<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width,initial-scale=1">
  <title>Ecosense AI — Saved Suggestions</title>

  <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}" />
</head>

<body>
  <!-- NAVIGATION -->
  <header class="nav small">
    <div class="nav-left">
      <span class="logo-icon">🌱</span>
      <div>
        <h1 class="brand">Ecosense AI</h1>
        <div class="tag">Your Sustainability Plan</div>
      </div>
    </div>

    <nav class="nav-right">
      <a href="/" class="nav-link">Home</a>
      <a href="/insights" class="nav-link">Results</a>
      <a href="/suggestions" class="nav-link active">Suggestions</a>
    </nav>
  </header>

  <!-- MAIN CONTENT -->
  <main class="main" style="grid-template-columns: 1fr; max-width: 850px; margin:auto;">

    <section class="card">

      <h2>Your Saved Recommendations</h2>
      <p class="small" style="margin-top:4px;">
        These suggestions help you reduce your carbon footprint over time.
      </p>

      <!-- If user has suggestions -->
      {% if ai and ai.recommendations %}
      <div class="ai-section" style="margin-top:20px;">
        <h3>Personalized Actions</h3>

        {% for r in ai.recommendations %}
        <div class="ai-reco">
          <div class="ai-title">{{ r.action }}</div>
          <div class="ai-body">{{ r.rationale }}</div>
          <div class="ai-meta">
            Estimated Savings: {{ r.est_weekly_savings_kg }} kg CO₂ / week •
            Difficulty: {{ r.difficulty }} •
            Timeframe: {{ r.timeframe }}
          </div>
        </div>
        {% endfor %}
      </div>

      {% else %}
      <!-- Fallback when no saved plan -->
      <div style="margin-top:20px;">
        <h3>No Saved Suggestions Yet</h3>
        <p class="muted">
          Your AI recommendations will appear here after completing the footprint analysis.
        </p>
        <a href="/" class="btn primary" style="margin-top:10px;">Calculate Now</a>
      </div>
      {% endif %}

      <!-- Monthly Plan -->
      <div class="plan" style="margin-top:25px;">
        <h3>🌿 Suggested Monthly Action Plan</h3>
        <p class="muted">A simple 4-week improvement plan.</p>

        <ol style="margin-top:10px; line-height:1.7;">
          <li>Week 1 — Replace 2 incandescent bulbs with LED.</li>
          <li>Week 2 — Add 3 plant-based meals.</li>
          <li>Week 3 — Start recycling and segregating waste.</li>
          <li>Week 4 — Track electricity usage decline.</li>
        </ol>
      </div>

      <!-- BUTTONS -->
      <div class="actions" style="margin-top:25px;">
        <button onclick="alert('PDF Export Coming Soon')" class="btn primary">Export as PDF</button>
        <button onclick="alert('Share Feature Coming Soon')" class="btn ghost">Share with Family</button>
      </div>

    </section>

  </main>

  <footer class="footer small">
    <div><a href="/">Back to Home</a></div>
  </footer>
</body>
</html>



Overwriting templates/suggestions.html


📘 8️⃣ Stylesheet (static/style.css)

The stylesheet adds:

- Green sustainability-themed UI  
- Modern gradient background  
- Responsive grid layout  
- Card-based design system  
- Styled charts and statistics  
- Interactive button and hover feedback  
- Light animations  
- Mobile optimizations  

Stored in:
static/style.css


In [ ]:
%%writefile static/style.css
/* -------------------------------------------------
   GLOBAL RESET + BASE
------------------------------------------------- */
* {
    margin: 0;
    padding: 0;
    box-sizing: border-box;
}
body {
    font-family: 'Inter', sans-serif;
    background: linear-gradient(135deg, #76b852 0%, #8dc26f 100%);
    min-height: 100vh;
    color: #1e2a1f;
    line-height: 1.5;
}

/* -------------------------------------------------
   NAVIGATION
------------------------------------------------- */
.nav {
    display: flex;
    justify-content: space-between;
    align-items: center;
    padding: 18px 28px;
    background: rgba(255, 255, 255, 0.85);
    backdrop-filter: blur(8px);
    box-shadow: 0 4px 15px rgba(0,0,0,0.08);
}

.nav.small {
    padding: 14px 24px;
}

.nav-left {
    display: flex;
    gap: 14px;
    align-items: center;
}

.logo-icon {
    font-size: 40px;
}

.brand {
    font-size: 24px;
    font-weight: 800;
    color: #2a4d14;
}

.tag {
    font-size: 13px;
    color: #567e3a;
    margin-top: -2px;
}

.nav-right {
    display: flex;
    gap: 20px;
}

.nav-link {
    text-decoration: none;
    font-weight: 600;
    color: #2e3d27;
    padding: 6px 10px;
    transition: 0.25s ease;
}

.nav-link.active {
    border-bottom: 2px solid #2e7d32;
}
.nav-link:hover {
    color: #1b5e20;
}

/* -------------------------------------------------
   PAGE LAYOUTS
------------------------------------------------- */
.main {
    width: 100%;
    padding: 30px;
    display: grid;
    justify-content: center;
}

/* home page uses two columns */
.main.home-layout {
    grid-template-columns: 1.1fr 0.9fr;
    gap: 25px;
}

/* results & suggestions use single column centered */
.main.single-column {
    max-width: 900px;
    margin: auto;
    grid-template-columns: 1fr;
}

/* -------------------------------------------------
   CARDS
------------------------------------------------- */
.card {
    background: rgba(255,255,255,0.97);
    padding: 25px;
    border-radius: 18px;
    box-shadow: 0 10px 25px rgba(0,0,0,0.08);
    backdrop-filter: blur(4px);
}

.card h2, .card h3 {
    color: #2c511b;
}

/* -------------------------------------------------
   FORM ELEMENTS
------------------------------------------------- */
.field {
    display: flex;
    flex-direction: column;
    gap: 6px;
    margin-bottom: 14px;
}

.field-label {
    font-weight: 600;
    font-size: 14px;
    color: #2e4e1a;
}

input, select {
    padding: 12px 14px;
    border-radius: 10px;
    background: #eef5e9;
    border: 2px solid #c7e6bf;
    font-size: 14px;
}

input:focus, select:focus {
    outline: none;
    border-color: #2e7d32;
    background: #f7fff2;
    box-shadow: 0 0 6px rgba(46,125,50,0.4);
}

.grid-2 {
    display: grid;
    grid-template-columns: 1fr 1fr;
    gap: 14px;
}

/* -------------------------------------------------
   BUTTON STYLES
------------------------------------------------- */
.btn {
    padding: 12px 20px;
    border-radius: 10px;
    border: none;
    cursor: pointer;
    font-weight: 700;
    font-size: 14px;
    transition: 0.25s ease;
}

.btn.primary {
    background: #2e7d32;
    color: #fff;
}
.btn.primary:hover {
    background: #205723;
}

.btn.ghost {
    background: transparent;
    border: 2px solid #2e7d32;
    color: #2e7d32;
}
.btn.ghost:hover {
    background: #2e7d32;
    color: #fff;
}

/* -------------------------------------------------
   BREAKDOWN STATS (results page)
------------------------------------------------- */
.breakdown-grid {
    display: grid;
    grid-template-columns: 1fr 1fr;
    gap: 14px;
    margin-top: 16px;
}

.stat {
    background: #f5f9f1;
    padding: 14px;
    border-radius: 12px;
    border-left: 5px solid #8cc769;
}

.stat-label {
    font-size: 14px;
    font-weight: 600;
}

.stat-value {
    font-size: 18px;
}

.total {
    font-size: 18px;
    margin-top: 20px;
}

.total-val {
    font-weight: 800;
    color: #29611d;
}

/* -------------------------------------------------
   CLASSIFICATION BOX
------------------------------------------------- */
.classification-box {
    margin-top: 22px;
    padding: 16px;
    border-radius: 12px;
}

.classification-box.low {
    background: #d6f5d0;
    border-left: 6px solid #1b5e20;
}

.classification-box.moderate {
    background: #fff4d1;
    border-left: 6px solid #ff9800;
}

.classification-box.high {
    background: #ffe0e0;
    border-left: 6px solid #c62828;
}

.class-title {
    font-weight: 700;
    font-size: 16px;
}
.class-msg {
    margin-top: 6px;
    font-size: 14px;
}

/* -------------------------------------------------
   AI RECOMMENDATIONS
------------------------------------------------- */
.ai-section {
    margin-top: 30px;
}

.ai-section h3 {
    margin-bottom: 10px;
}

.ai-reco {
    background: #ffffff;
    border-left: 5px solid #6bb04c;
    padding: 12px;
    border-radius: 10px;
    margin-bottom: 12px;
}

.ai-title {
    font-weight: 700;
}

.ai-meta {
    font-size: 12px;
    margin-top: 6px;
    color: #47663f;
}

/* -------------------------------------------------
   CHART CARD
------------------------------------------------- */
.chart-card {
    margin-top: 28px;
}
.chart-legend {
    display: flex;
    gap: 10px;
    flex-wrap: wrap;
    margin-top: 12px;
}

.lg-item {
    display: flex;
    align-items: center;
    gap: 6px;
    font-size: 12px;
}

.dot {
    width: 12px;
    height: 12px;
    border-radius: 50%;
}

/* -------------------------------------------------
   FOOTER
------------------------------------------------- */
.footer {
    text-align: center;
    padding: 20px;
    color: #22461e;
    margin-top: 40px;
}

/* -------------------------------------------------
   LOADING OVERLAY
------------------------------------------------- */
.loading-overlay {
    position: fixed;
    top:0; left:0;
    width:100%; height:100%;
    display:flex;
    flex-direction:column;
    justify-content:center;
    align-items:center;
    background:rgba(0,0,0,0.45);
    opacity:0; pointer-events:none;
    backdrop-filter:blur(4px);
    transition:0.2s ease;
}

.loading-overlay.active {
    opacity:1;
    pointer-events:all;
}

.spinner {
    width:50px; height:50px;
    border:4px solid #c8e6c9;
    border-top-color:#2e7d32;
    border-radius:50%;
    animation:spin 1s linear infinite;
}
@keyframes spin { to {transform:rotate(360deg);} }

#loadingText {
    color:white;
    font-weight:600;
    font-size:16px;
    margin-top:12px;
}

/* -------------------------------------------------
   RESPONSIVE
------------------------------------------------- */
@media (max-width: 900px) {
    .main.home-layout {
        grid-template-columns: 1fr;
    }
    .grid-2 {
        grid-template-columns: 1fr;
    }
}


Overwriting static/style.css


📘 9️⃣ Clean Startup — Kill Old Flask & ngrok

Before starting the app, previous Flask or ngrok sessions must be stopped:

!pkill -f flask || echo "No Flask process running"
!pkill -f ngrok || echo "No ngrok process running"

This avoids:
- Port conflicts  
- Duplicate servers  
- ngrok tunnel errors  


📘 🔟 Check Port 8000 for Conflicts

If Flask fails to launch, check which process is blocking port 8000:

!lsof -i :8000

Example output:
python3   1325   user   LISTEN  8000

Kill the process using:
!kill -9 1325


In [ ]:
# ===============================
# 6️⃣ Terminate any existing Flask or ngrok processes
# ===============================
# Ensures clean startup by killing previous instances
!pkill -f flask || echo "No Flask process running"
!pkill -f ngrok || echo "No ngrok process running"

^C
^C


In [ ]:
!lsof -i :8000

In [ ]:
!kill -9 1325

/bin/bash: line 1: kill: (1325) - No such process


📘 1️⃣1️⃣ Start Flask Server (Background Mode)

Run Flask seamlessly in the background:

!nohup python app.py > flask.log 2>&1 &

- Continues running even if the notebook moves to the next cell  
- All logs go to flask.log  


In [ ]:
# ===============================
# 7️⃣ Start the Flask server in the background
# ===============================
# Runs the app on port 8000 and logs output to flask.log
!nohup python app.py > flask.log 2>&1 &


📘 1️⃣2️⃣ Start ngrok Tunnel (Token Required)

ngrok exposes your Flask app publicly.

Your original token was removed.

Generate a new token:
https://dashboard.ngrok.com/get-started/your-authtoken

Replace inside notebook:
conf.get_default().auth_token = "YOUR_NGROK_TOKEN_HERE"

Start tunnel:
public_url = ngrok.connect(8000)
print(public_url)

You will get a link like:
https://abc123.ngrok-free.app

This is your LIVE Ecosense AI app link.


In [ ]:
# ===============================
# 8️⃣ Expose the Flask app using ngrok
# ===============================
# Replace the token with your own ngrok authentication token
from pyngrok import ngrok, conf
conf.get_default().auth_token = "YOUR_NGROK_TOKEN_HERE" # 🔑 Replace with your ngrok token

public_url = ngrok.connect(8000)
print("🌍 Public URL:", public_url)

# ===============================
# 9️⃣ Check Flask logs for debugging (optional)
# ===============================
# Displays the last 20 lines of the server log
!sleep 3 && tail -n 20 flask.log


🌍 Public URL: NgrokTunnel: "https://4c7e29da4eb0.ngrok-free.app" -> "http://localhost:8000"


📘 1️⃣3️⃣ View Flask Logs

Check backend activity and debug errors:

!tail -n 20 flask.log
!tail -n 50 flask.log

Useful for diagnosing:
- JSON parsing issues  
- Model loading failures  
- Route-related errors  


In [ ]:
!tail -n 50 flask.log

2025-11-21 12:41:32.228663: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763728892.248772    4954 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763728892.254683    4954 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763728892.270273    4954 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763728892.270310    4954 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763728892.270312    4954 computation_placer.cc:177] computation placer alr

In [ ]:
📘 1️⃣4️⃣ Project Complete
🎉 Ecosense AI is ready!

Your web app now supports:

- Accurate carbon calculations  
- AI-driven action planning  
- Session-based sustainability tracking  
- Scientific emission factor modeling  
- Real-time result visualization  
- Suggestion history retention  
- Clean UI & UX  

You can now integrate this into your company site or personal portfolio.


📘 1️⃣5️⃣ Learning Resources (Official References)

# 🌱 Learning Resources for Further Study

## Flask (Official Website)
https://flask.palletsprojects.com/

## Hugging Face Transformers
https://huggingface.co/docs/transformers/index

## Hugging Face Model Hub (Mistral Models)
https://huggingface.co/models?search=mistral

## BitsAndBytes (4-bit Quantization)
https://github.com/TimDettmers/bitsandbytes

## Accelerate (Efficient Hardware Utilization)
https://huggingface.co/docs/accelerate/index

## Chart.js (Used for Doughnut Charts)
https://www.chartjs.org/docs/latest/

## Pyngrok Documentation
https://pyngrok.readthedocs.io/en/latest/

## Carbon Footprint Standards (Reference)
https://ghgprotocol.org/
